# Preparación del entorno

In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=ea756958392fb809274983a7450770e72d387ab483c55e04123d6469d3b478d9
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 80.0 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.8.1
    Uninstalling duckdb-0.8.1:
      Successfully uninstalled duckdb-0.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.7/376.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.2 MB/s eta 0:00:00


In [3]:
import wget

url = "https://storage.googleapis.com/open-courses/dmeyf2023-8a1e/competencia_03_crudo.csv.gz"
ruta_destino = "/content/competencia_03_crudo.csv.gz"

wget.download(url, ruta_destino)

print("Archivo descargado en:", ruta_destino)

Archivo descargado en: /content/competencia_03_crudo.csv.gz


# Prueba con SQL

In [4]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////content/dmeyf.db

In [5]:
%%sql
create or replace table competencia_03 as
select
    *
from read_csv_auto('/content/competencia_03_crudo.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


# Construcción de Clase_ternaria

In [6]:
%sql select * from competencia_03 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,0.0,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,0.0,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,0.0,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,0.0,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,0.0,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66


In [7]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [8]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_03
), clientes as (
    select distinct numero_de_cliente from competencia_03
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , CASE
    WHEN (mes_0 = 1) AND (mes_1 = 1) AND (mes_2 = 1) THEN 'CONTINUA'
    WHEN (mes_0 = 1) AND (mes_1 = 0) AND (mes_2 = 1) THEN 'CONTINUA'
    WHEN (mes_0 = 1) AND (mes_1 = 0) AND (mes_2 = 0) THEN 'BAJA+1'
    WHEN (mes_0 = 1) AND (mes_1 = 1) AND (mes_2 = 0) THEN 'BAJA+2'
    ELSE NULL END AS clase_ternaria
    from todo t
    left join competencia_03 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [9]:
%%sql
alter table competencia_03 add column clase_ternaria VARCHAR(10)

,Success


In [10]:
%%sql
update competencia_03
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_03.numero_de_cliente = targets.numero_de_cliente and competencia_03.foto_mes = targets.foto_mes;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [11]:
%%sql
SELECT clase_ternaria, COUNT(*) AS count_clase
FROM targets
GROUP BY clase_ternaria;

,clase_ternaria,count_clase
0,None,330218
1,BAJA+2,21329
2,CONTINUA,4520714
3,BAJA+1,20767


In [12]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


# Descarga índice de inflación

## Armado del índice

In [13]:
!pip install pandas requests

In [14]:
import requests
from io import BytesIO

In [15]:
# URL del archivo
url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_08_23.xls"

# Descargar el archivo y cargar los datos de la hoja correcta
response = requests.get(url)
xls = pd.ExcelFile(BytesIO(response.content))

# Cargar los datos desde la hoja de interés
inflacion = pd.read_excel(xls, sheet_name="Índices IPC Cobertura Nacional", skiprows=5, usecols="B:CC")

In [16]:
inflacion

,2016-12-01,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,...,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,101.5859,103.6859,106.1476,108.9667,110.5301,111.8477,113.7852,115.3819,117.5719,...,1028.706,1079.2787,1134.5875,1202.979,1282.7091,1381.1601,1497.2147,1613.5895,1709.6115,1818.0838
4,100,101.3024,103.1605,106.0143,108.3515,109.7631,110.7483,112.0115,114.3467,116.4048,...,1096.5069,1135.001,1187.9252,1268.3385,1392.3999,1522.5337,1677.0441,1774.4119,1848.0332,1954.5101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Transponer los datos y establecer la tercera fila como encabezado
inflacion = inflacion.transpose()


In [18]:
inflacion = inflacion[3]

In [19]:
inflacion

2016-12-01          100
2017-01-01     101.5859
2017-02-01     103.6859
2017-03-01     106.1476
2017-04-01     108.9667
                ...    
2023-03-01    1381.1601
2023-04-01    1497.2147
2023-05-01    1613.5895
2023-06-01    1709.6115
2023-07-01    1818.0838
Name: 3, Length: 80, dtype: object

In [20]:
# Pasar a data frame
import pandas as pd

inflacion = pd.DataFrame(inflacion)

In [21]:
inflacion.index.name='foto_mes'

In [22]:
inflacion

,3
foto_mes,
2016-12-01,100
2017-01-01,101.5859
2017-02-01,103.6859
2017-03-01,106.1476
2017-04-01,108.9667
...,...
2023-03-01,1381.1601
2023-04-01,1497.2147
2023-05-01,1613.5895


In [23]:
# Renombrar la columna
inflacion.columns = ["indice_inflacion"]

# Eliminar las filas con valores nulos
inflacion.dropna(inplace=True)

# Eliminar guiones y los últimos dos dígitos de la columna "foto_mes"
inflacion.index = inflacion.index.astype(str).str.replace('-', '').str[:-2]

# Mostrar las primeras filas del DataFrame resultante
inflacion.head()

,indice_inflacion
foto_mes,
201612,100
201701,101.5859
201702,103.6859
201703,106.1476
201704,108.9667


## Inflación A Sql

In [24]:
# Guardar el DataFrame como un archivo CSV
inflacion.to_csv('/content/inflacion.csv')

In [25]:
%%sql
create or replace table inflacion as
select
    *
from read_csv_auto('/content/inflacion.csv')

,Success


In [26]:
%%sql
select *
from inflacion

,foto_mes,indice_inflacion
0,201612,100.0000
1,201701,101.5859
2,201702,103.6859
3,201703,106.1476
4,201704,108.9667
...,...,...
75,202303,1381.1601
76,202304,1497.2147
77,202305,1613.5895
78,202306,1709.6115


Agrego inflación acumulada hasta 202107 para hacer el drifting

In [27]:
%%sql
ALTER TABLE inflacion
ADD COLUMN indice_inflacion_acumulada FLOAT; -- Adjust the data type as needed

WITH InflacionAcumulada AS (
    SELECT
        foto_mes,
        indice_inflacion,
        (
            (SELECT indice_inflacion FROM inflacion WHERE foto_mes = '202107') / indice_inflacion
        ) AS inflacion_acumulada
    FROM
        inflacion
)
UPDATE inflacion
SET indice_inflacion_acumulada = (SELECT inflacion_acumulada FROM InflacionAcumulada WHERE inflacion.foto_mes = InflacionAcumulada.foto_mes);


,Count
0,80


In [28]:
%%sql
select *
from inflacion

,foto_mes,indice_inflacion,indice_inflacion_acumulada
0,201612,100.0000,4.980987
1,201701,101.5859,4.903227
2,201702,103.6859,4.803919
3,201703,106.1476,4.692510
4,201704,108.9667,4.571109
...,...,...,...
75,202303,1381.1601,0.360638
76,202304,1497.2147,0.332684
77,202305,1613.5895,0.308690
78,202306,1709.6115,0.291352


# Inner join competencia_03 e inflación

In [ ]:
%%sql
SELECT
    *
FROM
    competencia_03 c2
INNER JOIN
    inflacion i
ON
    c2.foto_mes = i.foto_mes;

In [ ]:
%%sql
SELECT
    *
FROM
    competencia_03 c2

# Solución del Data Drifting a variables monetarias

Primero, ajustos todas las variables monetarias por la inflación acumulada a 202107

In [ ]:
%%sql
SELECT
    Mrentabilidad * inflacion_acumulada AS Mrentabilidad_ajustada,
    mrentabilidad_annual * inflacion_acumulada AS mrentabilidad_annual_ajustada,
    mcomisiones * inflacion_acumulada AS mcomisiones_ajustada,
    mactivos_margen * inflacion_acumulada AS mactivos_margen_ajustada,
    mpasivos_margen * inflacion_acumulada AS mpasivos_margen_ajustada,
    mcuenta_corriente_adicional * inflacion_acumulada AS mcuenta_corriente_adicional_ajustada,
    mcuenta_corriente * inflacion_acumulada AS mcuenta_corriente_ajustada,
    mcaja_ahorro * inflacion_acumulada AS mcaja_ahorro_ajustada,
    mcaja_ahorro_adicional * inflacion_acumulada AS mcaja_ahorro_adicional_ajustada,
    mcaja_ahorro_dolares * inflacion_acumulada AS mcaja_ahorro_dolares_ajustada,
    mcuentas_saldo * inflacion_acumulada AS mcuentas_saldo_ajustada,
    mautoservicio * inflacion_acumulada AS mautoservicio_ajustada,
    mtarjeta_visa_consumo * inflacion_acumulada AS mtarjeta_visa_consumo_ajustada,
    mtarjeta_master_consumo * inflacion_acumulada AS mtarjeta_master_consumo_ajustada,
    mprestamos_personales * inflacion_acumulada AS mprestamos_personales_ajustada,
    mprestamos_prendarios * inflacion_acumulada AS mprestamos_prendarios_ajustada,
    mprestamos_hipotecarios * inflacion_acumulada AS mprestamos_hipotecarios_ajustada,
    mplazo_fijo_dolares * inflacion_acumulada AS mplazo_fijo_dolares_ajustada,
    mplazo_fijo_pesos * inflacion_acumulada AS mplazo_fijo_pesos_ajustada,
    minversion1_pesos * inflacion_acumulada AS minversion1_pesos_ajustada,
    minversion1_dolares * inflacion_acumulada AS minversion1_dolares_ajustada,
    minversion2 * inflacion_acumulada AS minversion2_ajustada,
    mpayroll * inflacion_acumulada AS mpayroll_ajustada,
    mpayroll2 * inflacion_acumulada AS mpayroll2_ajustada,
    mcuenta_debitos_automaticos * inflacion_acumulada AS mcuenta_debitos_automaticos_ajustada,
    mtarjeta_visa_debitos_automaticos * inflacion_acumulada AS mtarjeta_visa_debitos_automaticos_ajustada,
    mttarjeta_master_debitos_automaticos * inflacion_acumulada AS mttarjeta_master_debitos_automaticos_ajustada,
    mpagodeservicios * inflacion_acumulada AS mpagodeservicios_ajustada,
    mpagomiscuentas * inflacion_acumulada AS mpagomiscuentas_ajustada,
    mcajeros_propios_descuentos * inflacion_acumulada AS mcajeros_propios_descuentos_ajustada,
    mtarjeta_visa_descuentos * inflacion_acumulada AS mtarjeta_visa_descuentos_ajustada,
    mtarjeta_master_descuentos * inflacion_acumulada AS mtarjeta_master_descuentos_ajustada,
    mcomisiones_mantenimiento * inflacion_acumulada AS mcomisiones_mantenimiento_ajustada,
    mcomisiones_otras * inflacion_acumulada AS mcomisiones_otras_ajustada,
    mforex_buy * inflacion_acumulada AS mforex_buy_ajustada,
    mforex_sell * inflacion_acumulada AS mforex_sell_ajustada,
    mtransferencias_recibidas * inflacion_acumulada AS mtransferencias_recibidas_ajustada,
    mtransferencias_emitidas * inflacion_acumulada AS mtransferencias_emitidas_ajustada,
    mextraccion_autoservicio * inflacion_acumulada AS mextraccion_autoservicio_ajustada,
    mcheques_depositados * inflacion_acumulada AS mcheques_depositados_ajustada,
    mcheques_emitidos * inflacion_acumulada AS mcheques_emitidos_ajustada,
    mcheques_depositados_rechazados * inflacion_acumulada AS mcheques_depositados_rechazados_ajustada,
    mcheques_emitidos_rechazados * inflacion_acumulada AS mcheques_emitidos_rechazados_ajustada,
    matm * inflacion_acumulada AS matm_ajustada,
    matm_other * inflacion_acumulada AS matm_other_ajustada,
    Master_mfinanciacion_limite * inflacion_acumulada AS Master_mfinanciacion_limite_ajustada,
    Master_msaldototal * inflacion_acumulada AS Master_msaldototal_ajustada,
    Master_msaldopesos * inflacion_acumulada AS Master_msaldopesos_ajustada,
    Master_msaldodolares * inflacion_acumulada AS Master_msaldodolares_ajustada,
    Master_mconsumospesos * inflacion_acumulada AS Master_mconsumospesos_ajustada,
    Master_mconsumosdolares * inflacion_acumulada AS Master_mconsumosdolares_ajustada,
    Master_mlimitecompra * inflacion_acumulada AS Master_mlimitecompra_ajustada,
    Master_madelantopesos * inflacion_acumulada AS Master_madelantopesos_ajustada,
    Master_madelantodolares * inflacion_acumulada AS Master_madelantodolares_ajustada,
    Master_mpagado * inflacion_acumulada AS Master_mpagado_ajustada,
    Master_mpagospesos * inflacion_acumulada AS Master_mpagospesos_ajustada,
    Master_mpagosdolares * inflacion_acumulada AS Master_mpagosdolares_ajustada,
    Master_mconsumototal * inflacion_acumulada AS Master_mconsumototal_ajustada,
    Master_mpagominimo * inflacion_acumulada AS Master_mpagominimo_ajustado,
    Visa_mfinanciacion_limite * inflacion_acumulada AS Visa_mfinanciacion_limite_ajustada,
    Visa_msaldototal * inflacion_acumulada AS Visa_msaldototal_ajustada,
    Visa_msaldopesos * inflacion_acumulada AS Visa_msaldopesos_ajustada,
    Visa_msaldodolares * inflacion_acumulada AS Visa_msaldodolares_ajustada,
    Visa_mconsumospesos * inflacion_acumulada AS Visa_mconsumospesos_ajustada,
    Visa_mconsumosdolares * inflacion_acumulada AS Visa_mconsumosdolares_ajustada,
    Visa_mlimitecompra * inflacion_acumulada AS Visa_mlimitecompra_ajustada,
    Visa_madelantopesos * inflacion_acumulada AS Visa_madelantopesos_ajustada,
    Visa_madelantodolares * inflacion_acumulada AS Visa_madelantodolares_ajustada,
    Visa_mpagado * inflacion_acumulada AS Visa_mpagado_ajustada,
    Visa_mpagospesos * inflacion_acumulada AS Visa_mpagospesos_ajustada,
    Visa_mpagosdolares * inflacion_acumulada AS Visa_mpagosdolares_ajustada,
    Visa_mconsumototal * inflacion_acumulada AS Visa_mconsumototal_ajustada,
    Visa_mpagominimo * inflacion_acumulada AS Visa_mpagominimo_ajustado
FROM
    competencia_03;

Después,ya elimino la variable "inflación_acumulada" ya que no es de más utilidad, y las variables monetarias nominales

In [ ]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN inflacion_acumulada;

In [ ]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN Mrentabilidad,
DROP COLUMN mrentabilidad_annual,
DROP COLUMN mcomisiones,
DROP COLUMN mactivos_margen,
DROP COLUMN mpasivos_margen,
DROP COLUMN mcuenta_corriente_adicional,
DROP COLUMN mcuenta_corriente,
DROP COLUMN mcaja_ahorro,
DROP COLUMN mcaja_ahorro_adicional,
DROP COLUMN mcaja_ahorro_dolares,
DROP COLUMN mcuentas_saldo,
DROP COLUMN mautoservicio,
DROP COLUMN mtarjeta_visa_consumo,
DROP COLUMN mtarjeta_master_consumo,
DROP COLUMN mprestamos_personales,
DROP COLUMN mprestamos_prendarios,
DROP COLUMN mprestamos_hipotecarios,
DROP COLUMN mplazo_fijo_dolares,
DROP COLUMN mplazo_fijo_pesos,
DROP COLUMN minversion1_pesos,
DROP COLUMN minversion1_dolares,
DROP COLUMN minversion2,
DROP COLUMN mpayroll,
DROP COLUMN mpayroll2,
DROP COLUMN mcuenta_debitos_automaticos,
DROP COLUMN mtarjeta_visa_debitos_automaticos,
DROP COLUMN mttarjeta_master_debitos_automaticos,
DROP COLUMN mpagodeservicios,
DROP COLUMN mpagomiscuentas,
DROP COLUMN mcajeros_propios_descuentos,
DROP COLUMN mtarjeta_visa_descuentos,
DROP COLUMN mtarjeta_master_descuentos,
DROP COLUMN mcomisiones_mantenimiento,
DROP COLUMN mcomisiones_otras,
DROP COLUMN mforex_buy,
DROP COLUMN mforex_sell,
DROP COLUMN mtransferencias_recibidas,
DROP COLUMN mtransferencias_emitidas,
DROP COLUMN mextraccion_autoservicio,
DROP COLUMN mcheques_depositados,
DROP COLUMN mcheques_emitidos,
DROP COLUMN mcheques_depositados_rechazados,
DROP COLUMN mcheques_emitidos_rechazados,
DROP COLUMN matm,
DROP COLUMN Master_mfinanciacion_limite,
DROP COLUMN Master_msaldototal,
DROP COLUMN Master_msaldopesos,
DROP COLUMN Master_msaldodolares,
DROP COLUMN Master_mconsumospesos,
DROP COLUMN Master_mconsumosdolares,
DROP COLUMN Master_mlimitecompra,
DROP COLUMN Master_madelantopesos,
DROP COLUMN Master_madelantodolares,
DROP COLUMN Master_mpagado,
DROP COLUMN Master_mpagospesos,
DROP COLUMN Master_mpagosdolares,
DROP COLUMN Master_mconsumototal,
DROP COLUMN Master_mpagominimo,
DROP COLUMN Visa_mfinanciacion_limite,
DROP COLUMN Visa_msaldototal,
DROP COLUMN Visa_msaldopesos,
DROP COLUMN Visa_msaldodolares,
DROP COLUMN Visa_mconsumospesos,
DROP COLUMN Visa_mconsumosdolares,
DROP COLUMN Visa_mlimitecompra,
DROP COLUMN Visa_madelantopesos,
DROP COLUMN Visa_madelantodolares,
DROP COLUMN Visa_mpagado,
DROP COLUMN Visa_mpagospesos,
DROP COLUMN Visa_mpagosdolares,
DROP COLUMN Visa_mconsumototal,
DROP COLUMN Visa_mpagominimo;

# Guardado

In [ ]:
%%sql
copy competencia_03 to '/content/competencia_03.csv.gz' (FORMAT CSV, HEADER)